In [96]:
import pandas as pd
import matplotlib
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.impute import SimpleImputer
from sklearn.tree import DecisionTreeClassifier

circuits = pd.read_csv('data/circuits.csv')
constructor_results = pd.read_csv('data/constructor_results.csv')
constructor_standings = pd.read_csv('data/constructor_standings.csv')
constructors = pd.read_csv('data/constructors.csv')
driver_standings = pd.read_csv('data/driver_standings.csv')
drivers = pd.read_csv('data/drivers.csv')
lap_times = pd.read_csv('data/lap_times.csv')
pit_stops = pd.read_csv('data/pit_stops.csv')
qualifying = pd.read_csv('data/qualifying.csv')
races = pd.read_csv('data/races.csv')
results = pd.read_csv('data/results.csv')
seasons = pd.read_csv('data/seasons.csv')
sprint_results = pd.read_csv('data/sprint_results.csv')
status = pd.read_csv('data/status.csv')

In [97]:
races_cleaned = races[["raceId", "year", "round", "circuitId"]].copy()
# print(races_cleaned)

races_cleaned = races_cleaned.sort_values(by=['year', 'round'])

races_cleaned = races_cleaned[races_cleaned["year"] >= 2000]

results_cleaned = results[["raceId", "driverId", "constructorId", "grid", "positionOrder"]].copy()
# print(results_cleaned)

df = pd.merge(races_cleaned, results_cleaned, on='raceId')
df['Top 3 Finish'] = df['positionOrder'].le(3).astype(int)
 
# print(results_cleaned)
# print(pit_stops)
print(constructor_standings)

       constructorStandingsId  raceId  constructorId  points  position  \
0                           1      18              1    14.0         1   
1                           2      18              2     8.0         3   
2                           3      18              3     9.0         2   
3                           4      18              4     5.0         4   
4                           5      18              5     2.0         5   
...                       ...     ...            ...     ...       ...   
13046                   28568    1110            214    57.0         6   
13047                   28569    1110              3    11.0         7   
13048                   28570    1110            213     3.0        10   
13049                   28571    1110            210    11.0         8   
13050                   28572    1110              1   103.0         5   

      positionText  wins  
0                1     1  
1                3     0  
2                2     0  
3  

In [98]:
df.describe().T

,count,mean,std,min,25%,50%,75%,max
raceId,9400.0,598.320426,431.981674,1.0,114.00,866.0,986.0,1110.0
year,9400.0,2011.647447,6.725949,2000.0,2006.00,2012.0,2017.0,2023.0
round,9400.0,9.877872,5.538708,1.0,5.00,10.0,14.0,22.0
circuitId,9400.0,18.957234,20.474229,1.0,6.00,13.0,21.0,79.0
driverId,9400.0,323.793191,388.235245,1.0,14.75,35.0,821.0,858.0
constructorId,9400.0,45.602553,72.399018,1.0,4.00,9.0,21.0,214.0
grid,9400.0,10.970319,6.207779,0.0,6.00,11.0,16.0,24.0
positionOrder,9400.0,11.111702,6.179882,1.0,6.00,11.0,16.0,24.0
Top 3 Finish,9400.0,0.142021,0.349091,0.0,0.00,0.0,0.0,1.0


In [99]:
# Calculate driver and constructor averages
driver_avg_finish = results.groupby('driverId')['positionOrder'].mean().reset_index().rename(columns={'positionOrder': 'driverAvgFinish'})
constructor_avg_finish = results.groupby('constructorId')['positionOrder'].mean().reset_index().rename(columns={'positionOrder': 'constructorAvgFinish'})

# Calculate win rates and podium finishes
results['win'] = (results['positionOrder'] == 1).astype(int)
results['podium'] = (results['positionOrder'] <= 3).astype(int)

driver_win_rate = results.groupby('driverId')['win'].mean().reset_index().rename(columns={'win': 'driverWinRate'})
driver_podium_rate = results.groupby('driverId')['podium'].mean().reset_index().rename(columns={'podium': 'driverPodiumRate'})

constructor_win_rate = results.groupby('constructorId')['win'].mean().reset_index().rename(columns={'win': 'constructorWinRate'})
constructor_podium_rate = results.groupby('constructorId')['podium'].mean().reset_index().rename(columns={'podium': 'constructorPodiumRate'})

# Assuming pit_stop_df is your DataFrame containing the pit stop data
average_pit_stop_duration = pit_stops.groupby(['raceId', 'driverId'])['milliseconds'].mean().reset_index().rename(columns={'milliseconds': 'averagePitStopDuration'})

constructor_standings_relevant = constructor_standings[['raceId', 'constructorId', 'points', 'position', 'wins']]

# Merge these features back into the main DataFrame
df = df.merge(driver_avg_finish, on='driverId', how='left')
df = df.merge(constructor_avg_finish, on='constructorId', how='left')
df = df.merge(driver_win_rate, on='driverId', how='left')
df = df.merge(driver_podium_rate, on='driverId', how='left')
df = df.merge(constructor_win_rate, on='constructorId', how='left')
df = df.merge(constructor_podium_rate, on='constructorId', how='left')
df = pd.merge(df, average_pit_stop_duration, on=['raceId', 'driverId'], how='left')

df = pd.merge(df, constructor_standings_relevant, on=['raceId', 'constructorId'], how='left', suffixes=('', '_const'))


print(df)

      raceId  year  round  circuitId  driverId  constructorId  grid  \
0        158  2000      1          1        30              6     3   
1        158  2000      1          1        22              6     4   
2        158  2000      1          1        23              3    11   
3        158  2000      1          1        35             16     8   
4        158  2000      1          1        21             22     9   
...      ...   ...    ...        ...       ...            ...   ...   
9395    1110  2023     12         13       817            213    19   
9396    1110  2023     12         13       858              3    18   
9397    1110  2023     12         13       807            210     0   
9398    1110  2023     12         13       832              6     4   
9399    1110  2023     12         13       857              1     5   

      positionOrder  Top 3 Finish  driverAvgFinish  constructorAvgFinish  \
0                 1             1         6.879870              8.33867

In [102]:
# Step 1: Drop duplicate and unnecessary '_const' columns
columns_to_drop = ['points_const', 'position_const', 'wins_const']
df.drop(columns=columns_to_drop, inplace=True, errors='ignore')

# Step 2: Handle missing values for 'averagePitStopDuration'
# If missing values represent races without pit stops, you might fill these with a default value or drop them.
# Filling with a high value (assuming longer duration means no pit stop) or the median if you're keeping the row.
df['averagePitStopDuration'] = df['averagePitStopDuration'].fillna(df['averagePitStopDuration'].median())

# Handling minimal missing values in 'points', 'position', and 'wins'
# Assuming these missing values can be filled with zeros (no points, position out of standing, no wins)
for col in ['points', 'position', 'wins']:
    df[col] = df[col].fillna(0)

# Verify the clean-up
print(df)


      raceId  year  round  circuitId  driverId  constructorId  grid  \
0        158  2000      1          1        30              6     3   
1        158  2000      1          1        22              6     4   
2        158  2000      1          1        23              3    11   
3        158  2000      1          1        35             16     8   
4        158  2000      1          1        21             22     9   
...      ...   ...    ...        ...       ...            ...   ...   
9395    1110  2023     12         13       817            213    19   
9396    1110  2023     12         13       858              3    18   
9397    1110  2023     12         13       807            210     0   
9398    1110  2023     12         13       832              6     4   
9399    1110  2023     12         13       857              1     5   

      positionOrder  Top 3 Finish  driverAvgFinish  constructorAvgFinish  \
0                 1             1         6.879870              8.33867

In [103]:
df.to_csv('cleaned.csv', index=False)